# Homework 4 - MapReduce

### DUE: 03/02/2017 before class at 10:30am

In this homework, we are practicing the MapReduce programming paradigm. 

You are required to turn in this notebook as BDM\_HW4\_**NetId**.ipynb. You will be asked to complete each task using the accompanied *mapreduce* package (as **mapreduce.py**) using one or more MapReduce "jobs". For each such job (with mr.run()), you are expected to supply a mapper and a reducer as needed. Below are sample usage of the package:

```python
    # Run on input1 using your mapper1 and reducer1 function
    output = list(mr.run(input1, mapper1, reducer1))

    # Run on input2 using only your mapper2, no reduce phase
    output = list(mr.run(input2, mapper2))
    
    # Run on input3 using 2 nested MapReduce jobs
    output = list(mr.run(mr.run(input3, mapper3, reducer3), mapper4, reducer4))
```
    
Please note that the output the mr.run() is always a **generator**. You have to cast it to a list if you'd like to view, index or print it out.

**NOTE:** if you are using Python3 (though I still encourage you to use Python2 for this class), please download the file *mapreduce.py* from NYU Classes again.

## Task (10 points)

There is only one task in this homework. You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [21]:
import csv 
import mapreduce as mr

def mapper1(row):        
    key, value = (row['From'], row['To'].split(';'))
    for i in value:
        yield key, i
    
def reducer1(k2v2):
    sender, union = k2v2
    return (sender,sorted(list(set(union))))

def mapper2(row):
    key, value = (row['From'], row['To'].split(';'))
    for i in value:
        yield i, key
        
def reducer2(k3v3):
    receiver, union = k3v3
    return (receiver,list(set(union)))

def IsReciprocal(sender, recipientlist):
    if recipientlist is not None:
        for r in recipientlist:
            if sender == r:
                return (sender, "reciprocal")
    else:
        return (sender, "directed")

def mapper3(row):
    sender, sl, rl = row[0], row[1], row[2] #sl, rl = lists
    for s in sl:
        yield (sender, IsReciprocal(s, rl))

        
    
def mapper4(k1v1):
    k1, v1 = k1v1
    if v1 != None:
        if v1[1] == 'reciprocal':
            if k1 != v1[0]:
                if k1.endswith('enron.com'):
                    if v1[0].endswith('enron.com'):
                        yield (v1[1], tuple(sorted(tuple((k1, v1[0])))))

def reducer4(k2v2):
    k2, v2 = k2v2
    return set(v2)
    
    
def mapper5(row):
    return 'recipropcal', '{} : {}'.format(row[0], row[1])

    
with open('enron_mails_small.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output = list(mr.run (reader, mapper1, reducer1))
    
with open('enron_mails_small.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output2 = list(mr.run(reader,mapper2, reducer2))
    

dict1 = dict(output)
dict2 = dict(output2)
lst3 = [(k, dict1.get(k), dict2.get(k)) for k in sorted(dict1)]

output3 = list(mr.run(lst3,mapper3))   
output4 = list(mr.run(output3, mapper4))

output4 = list(mr.run(output3, mapper4,reducer4))
output4 = list(map(mapper5,list(output4[0])))

In [22]:
output4

[('recipropcal', 'debra.perlingiere@enron.com : kevin.ruscitti@enron.com'),
 ('recipropcal', 'carol.clair@enron.com : debra.perlingiere@enron.com'),
 ('recipropcal', 'eric.bass@enron.com : susan.scott@enron.com'),
 ('recipropcal', 'brenda.whitehead@enron.com : elizabeth.sager@enron.com'),
 ('recipropcal', 'elizabeth.sager@enron.com : richard.sanders@enron.com'),
 ('recipropcal', 'richard.sanders@enron.com : sara.shackleton@enron.com'),
 ('recipropcal', 'mark.haedicke@enron.com : michelle.cash@enron.com'),
 ('recipropcal', 'fletcher.sturm@enron.com : greg.whalley@enron.com'),
 ('recipropcal', 'shirley.crenshaw@enron.com : vince.kaminski@enron.com'),
 ('recipropcal', 'drew.fossum@enron.com : susan.scott@enron.com'),
 ('recipropcal', 'mark.haedicke@enron.com : richard.sanders@enron.com'),
 ('recipropcal', 'elizabeth.sager@enron.com : mark.taylor@enron.com'),
 ('recipropcal', 'fletcher.sturm@enron.com : sally.beck@enron.com'),
 ('recipropcal',
  'pinnamaneni.krishnarao@enron.com : vince.ka